In [1]:
!pip install mysql-connector-python

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np  # Import numpy for mean calculation
import mysql.connector as msc

conn = msc.connect(
    host="localhost",
    user="root",
    password="wasd",
    database="ir_policy"
)

# SOTA Performance 

In [32]:
cursor1 = conn.cursor()
cursor1.execute("SELECT * FROM ir_policy.apps_data")
data1 = cursor1.fetchall() 
cursor1.close() 

cursor2 = conn.cursor()
cursor2.execute("SELECT * FROM ir_policy.generated_data")
data2 = cursor2.fetchall()  
cursor2.close() 

True

In [33]:
privacy_policy_texts = [data[3] for data in data1] 
generated_data_texts = [data[1] for data in data2]  

print(privacy_policy_texts)

['Skip to content\nWhatsApp Main Page\n\nFeatures\nPrivacy\nHelp Center\nBlog\nWhatsApp Web\nDownload\nLast modified: January 4, 2021 (archived versions)\n\nWhatsApp Privacy Policy\nIf you live in the European Region, WhatsApp Ireland Limited provides the services to you under this Terms of Service and Privacy Policy.\n\nWhatsApp Legal Info\nIf you live outside the European Region, WhatsApp LLC ("WhatsApp," "our," "we," or "us") provides our Services to you under this Terms of Service and Privacy Policy.\n\nOur Privacy Policy ("Privacy Policy") helps explain our data practices, including the information we process to provide our Services.\n\nFor example, our Privacy Policy talks about what information we collect and how this affects you. It also explains the steps we take to protect your privacy, like building our Services so delivered messages aren’t stored by us and giving you control over who you communicate with on our Services.\n\nWe are one of the Meta Companies. You can learn mo

In [34]:
print(generated_data_texts)

["This Privacy Policy applies to all users of WhatsApp in the European Region, WhatsApp Ireland Limited provides the services to you under this Terms of Service and Privacy Policy, and WhatsApp is a member of the Meta family of companies. WhatsApp's Privacy Policy explains our data practices, including the information we process to provide our Services to you. It also explains the steps we take to protect your privacy, like building our Services so delivered messages aren't stored by us and giving you control over who you communicate with on our Services. For example, we've added more information about more recent product features and functionalities, how we process your data for safety, security, and integrity, and added more direct links to user settings, Help Center articles, and how you can manage your information. The following scenarios describe circumstances where we may store your messages in the course of delivering them: Undelivered Messages. When a user forwards media within

- Cosine Similarity

In [35]:
vectorizer = TfidfVectorizer()
generated_data_vectors = vectorizer.fit_transform(generated_data_texts)
privacy_policy_vectors = vectorizer.transform(privacy_policy_texts) 

cosine_similarities = cosine_similarity(generated_data_vectors, privacy_policy_vectors)

average_cosine_similarity = np.mean(cosine_similarities.diagonal())

for app_id, cosine_sim in zip([data[0] for data in data1], cosine_similarities.diagonal()):
    print(f"App ID {app_id}: Cosine Similarity = {cosine_sim:.2f}")

print(f"Average Cosine Similarity: {average_cosine_similarity:.2f}")

cursor1.close()
cursor2.close()
conn.close()

App ID 1: Cosine Similarity = 0.71
App ID 2: Cosine Similarity = 0.75
App ID 3: Cosine Similarity = 0.68
App ID 4: Cosine Similarity = 0.66
App ID 5: Cosine Similarity = 0.59
App ID 6: Cosine Similarity = 0.73
App ID 7: Cosine Similarity = 0.75
App ID 8: Cosine Similarity = 0.77
App ID 9: Cosine Similarity = 0.84
App ID 10: Cosine Similarity = 0.82
App ID 11: Cosine Similarity = 0.87
App ID 12: Cosine Similarity = 0.72
App ID 13: Cosine Similarity = 0.82
App ID 14: Cosine Similarity = 0.82
App ID 15: Cosine Similarity = 0.77
App ID 16: Cosine Similarity = 0.78
App ID 17: Cosine Similarity = 0.72
App ID 18: Cosine Similarity = 0.73
App ID 19: Cosine Similarity = 0.79
App ID 20: Cosine Similarity = 0.78
App ID 21: Cosine Similarity = 0.71
App ID 22: Cosine Similarity = 0.77
App ID 23: Cosine Similarity = 0.68
App ID 24: Cosine Similarity = 0.62
App ID 25: Cosine Similarity = 0.84
App ID 26: Cosine Similarity = 0.84
App ID 27: Cosine Similarity = 0.84
App ID 28: Cosine Similarity = 0.83
A

- ROUGE SCORES

In [36]:
from rouge import Rouge

rouge = Rouge()

aggregate_scores = {'rouge-1': {'f': [], 'p': [], 'r': []},
                    'rouge-2': {'f': [], 'p': [], 'r': []},
                    'rouge-l': {'f': [], 'p': [], 'r': []}}

for app_id, generated_text, reference_text in zip([data[0] for data in data1], generated_data_texts, privacy_policy_texts):
    scores = rouge.get_scores(generated_text, reference_text)[0]
    
    for key in aggregate_scores.keys():
        aggregate_scores[key]['f'].append(scores[key]['f'])
        aggregate_scores[key]['p'].append(scores[key]['p'])
        aggregate_scores[key]['r'].append(scores[key]['r'])
        
    print(f"App ID {app_id} ROUGE scores:")
    for rouge_key, rouge_scores in scores.items():
        print(f"{rouge_key}: f({rouge_scores['f']:.2f}), p({rouge_scores['p']:.2f}), r({rouge_scores['r']:.2f})")

print("Aggregate ROUGE scores:")
for rouge_key, score_lists in aggregate_scores.items():
    f_mean = np.mean(score_lists['f'])
    p_mean = np.mean(score_lists['p'])
    r_mean = np.mean(score_lists['r'])
    print(f"{rouge_key}: f({f_mean:.2f}), p({p_mean:.2f}), r({r_mean:.2f})")

App ID 1 ROUGE scores:
rouge-1: f(0.35), p(0.84), r(0.22)
rouge-2: f(0.22), p(0.67), r(0.13)
rouge-l: f(0.35), p(0.83), r(0.22)
App ID 2 ROUGE scores:
rouge-1: f(0.22), p(0.91), r(0.13)
rouge-2: f(0.10), p(0.70), r(0.06)
rouge-l: f(0.22), p(0.89), r(0.13)
App ID 3 ROUGE scores:
rouge-1: f(0.31), p(0.91), r(0.19)
rouge-2: f(0.18), p(0.84), r(0.10)
rouge-l: f(0.31), p(0.90), r(0.18)
App ID 4 ROUGE scores:
rouge-1: f(0.27), p(0.84), r(0.16)
rouge-2: f(0.16), p(0.71), r(0.09)
rouge-l: f(0.27), p(0.83), r(0.16)
App ID 5 ROUGE scores:
rouge-1: f(0.26), p(0.80), r(0.16)
rouge-2: f(0.14), p(0.67), r(0.08)
rouge-l: f(0.26), p(0.80), r(0.16)
App ID 6 ROUGE scores:
rouge-1: f(0.26), p(0.90), r(0.15)
rouge-2: f(0.14), p(0.74), r(0.08)
rouge-l: f(0.25), p(0.88), r(0.15)
App ID 7 ROUGE scores:
rouge-1: f(0.22), p(0.98), r(0.12)
rouge-2: f(0.13), p(0.93), r(0.07)
rouge-l: f(0.22), p(0.97), r(0.12)
App ID 8 ROUGE scores:
rouge-1: f(0.41), p(0.75), r(0.28)
rouge-2: f(0.28), p(0.59), r(0.18)
rouge-l: f(

- BERT SCORES

In [38]:
!pip install bert-score

In [40]:
from bert_score import score

candidates = generated_data_texts
references = privacy_policy_texts

P, R, F1 = score(candidates, references, lang='en', verbose=True)

for app_id, p, r, f1 in zip([data[0] for data in data1], P, R, F1):
    print(f"App ID {app_id}: Precision = {p:.4f}, Recall = {r:.4f}, F1 Score = {f1:.4f}")

average_precision = P.mean().item()
average_recall = R.mean().item()
average_f1 = F1.mean().item()

print(f"Average BERTScore Precision: {average_precision:.4f}")
print(f"Average BERTScore Recall: {average_recall:.4f}")
print(f"Average BERTScore F1: {average_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 226.42 seconds, 0.21 sentences/sec
App ID 1: Precision = 0.8531, Recall = 0.8503, F1 Score = 0.8517
App ID 2: Precision = 0.8364, Recall = 0.8346, F1 Score = 0.8355
App ID 3: Precision = 0.8665, Recall = 0.8692, F1 Score = 0.8679
App ID 4: Precision = 0.8784, Recall = 0.8750, F1 Score = 0.8767
App ID 5: Precision = 0.8604, Recall = 0.8631, F1 Score = 0.8617
App ID 6: Precision = 0.8444, Recall = 0.8019, F1 Score = 0.8226
App ID 7: Precision = 0.8880, Recall = 0.8714, F1 Score = 0.8796
App ID 8: Precision = 0.8395, Recall = 0.8294, F1 Score = 0.8344
App ID 9: Precision = 0.8707, Recall = 0.8583, F1 Score = 0.8644
App ID 10: Precision = 0.8792, Recall = 0.8552, F1 Score = 0.8670
App ID 11: Precision = 0.8768, Recall = 0.8706, F1 Score = 0.8737
App ID 12: Precision = 0.8512, Recall = 0.8331, F1 Score = 0.8420
App ID 13: Precision = 0.8876, Recall = 0.8813, F1 Score = 0.8844
App ID 14: Precision = 0.8548, Recall = 0.8406, F1 Score = 0.8476
App ID 15: Precision = 0.8697, Recall = 0.